## Local Setup

If you prefer to work locally, see the following instructions for setting up Python in a virtual environment.
You can then ignore the instructions in "Colab Setup".

If you haven't yet, create a [conda](https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html) environment using:
```
conda create --name rl_exercises
conda activate rl_exercises
```
Torch recommends installation using conda rather than pip, so run e.g.:
```
conda install pytorch pytorch-cuda=11.8 -c pytorch -c nvidia
```
For this exercise, you require a CUDA-enabled GPU, as training an image-based model on the CPU takes a very long time.
Visit [the installation page](https://pytorch.org/get-started/locally/) to see the options available for different CUDA versions.
The remaining dependencies can be installed with pip:
```
pip install matplotlib numpy tqdm ipykernel "gymnasium[classic-control, other]" scikit-learn
```

Even if you are running the Jupyter notebook locally, please run the code cells in **Colab Setup**, because they define some global variables required later.

## Colab Setup

Google Colab provides you with a temporary environment for python programming.
While this conveniently works on any platform and internally handles dependency issues and such, it also requires you to set up the environment from scratch every time.
The "Colab Setup" section below will be part of **every** exercise and contains utility that is needed before getting started.

**IMPORTANT**: For this exercise, you require a GPU runtime environment, as training an image-based model on the CPU takes a very long time.
To do this, select "Change runtime type" from the context menu in the top right corner (next to the **Connect** button), and select **T4 GPU**.

There is a timeout of about ~12 hours with Colab while it is active (and less if you close your browser window).
Any changes you make to the Jupyter notebook itself should be saved to your Google Drive.
We also save all recordings and logs in it by default so that you won't lose your work in the event of an instance timeout.
However, you will need to re-mount your Google Drive and re-install packages with every new instance.

In [1]:
"""Your work will be stored in a folder called `rl_ws23` by default to prevent Colab
instance timeouts from deleting your edits.
We do this by mounting your google drive on the virtual machine created in this colab
session. For this, you will likely need to sign in to your Google account and allow
access to your Google Drive files.
"""

from pathlib import Path

try:
    from google.colab import drive

    drive.mount("/content/gdrive")
    COLAB = True
except ImportError:
    COLAB = False

# Create paths in your google drive
if COLAB:
    DATA_ROOT = Path("/content/gdrive/My Drive/rl_ws23")
    DATA_ROOT.mkdir(parents=True, exist_ok=True)

    DATA_ROOT_STR = str(DATA_ROOT)
    %cd "$DATA_ROOT"
else:
    DATA_ROOT = Path.cwd() / "rl_ws23"

# Install python packages
if COLAB:
    %pip install matplotlib numpy tqdm "gymnasium[atari, accept-rom-license, classic-control, other]"  scikit-learn

Mounted at /content/gdrive
/content/gdrive/My Drive/rl_ws23
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 38.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.0 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=88a0ea299288202ca52e1b765998964f0efe5a1e4c702be04eebde915866fa10
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license


# Exercise 4 - Model-Based Reinforcement Learning

Designed by Vaisakh Shaj and Niklas Freymuth.

In this homework, we are going to implement a simple model predictive control (MPC) algorithm using a learned dynamics and reward model.

### Principle
We consider the optimal control problem of an MDP with an **unknown deterministic** reward function $r$ and subject to **unknown deterministic** dynamics $s_{t+1} = f(s_t, a_t)$:

$$\max_{(a_0,a_1,\dotsc)} \sum_{t=0}^\infty \gamma^t r(s_t,a_t)$$

In **model-based reinforcement learning**, this problem is solved in **two steps**:
1. **Model learning**:
We learn a model of the dynamics $f_\theta \simeq f$ and reward function $r_\theta \simeq r$ through regression on interaction data.
2. **Planning**:
We leverage the dynamics model $f_\theta$ to compute the optimal trajectory $$\max_{(a_0,a_1,\dotsc)} \sum_{t=0}^\infty \gamma^t r_{\theta}(\hat{s}_t,a_t)$$ following the learnt dynamics $\hat{s}_{t+1} = f_\theta(\hat{s}_t, a_t)$.

(We can easily extend to stochastic dynamics, but we consider the simpler case in this homework)


In this homework you will implement the model-based algorithm proposed in section IV of [this paper](https://arxiv.org/abs/1708.02596) with some differences:

1. along with the next environment state, also the reward is learned. To do that another neural network has been used.
2. We train on the pybullet gym environment of inverted pendulum.

$$
\begin{aligned}
&\overline{\text { Algorithm } \mathbf{1} \text { Model-based Reinforcement Learning }} \\
&\hline \text { 1: gather dataset } \mathcal{D}_{\text {RAND }} \text { of random trajectories } \\
&\text { 2: initialize empty dataset } \mathcal{D}_{\text {RL }} \text {, and randomly initialize } f_{\theta} \\
&  \text{ 3: } \textbf{for} \text{ iter=1 to max_iter } \textbf{do}  \\
&\quad  \quad \quad \textbf {Model Learning } \\
&\text { 4:} \quad  \quad \text{ train } f_{\theta}(\mathbf{s}, \mathbf{a}) \text{ and } r_{\theta}(\mathbf{s}, \mathbf{a}) \text { by performing gradient descent on MSE Loss }  \\
& \quad \quad  \quad \text { using } \mathcal{D}_{\text {RAND }} \text { and } \mathcal{D}_{\text {RL }} \\
&\quad  \quad \quad \textbf {Planning and More Experience Collection } \\
&\text { 5: } \quad  \quad  \textbf { for } t=1 \text { to } T \textbf { do } \\
&\text { 6: } \quad \quad \quad \quad \text { get agent's current state } \mathbf{s}_{t} \\
&\text { 7: } \quad \quad \quad \quad \text { use } f_{\theta} \text { and } r_{\theta} \text { to estimate optimal action sequence } \mathbf{A}_{t}^{(H)} \\
&\text { 8: } \quad \quad \quad \quad \text { execute first action } \mathbf{a}_{t} \text { from selected action sequence } \\
&\text { 9: } \quad \quad \quad \quad\text { Add } \{s_t,  s_{t+1}, r_t, a_t\} \text { to } \mathcal{D}_{\text {RL }} \\
&\text { 10: } \quad \text { end for } \\
&\text { 11: end for } \\
&\hline
\end{aligned}
$$

You will notice that compared to model free methods, model based methods are more sample efficient, at the cost of a more complex algorithm.

### Code Overview

The code below is organized into the following blocks.
The bolded sections require your input:
 * Import statements and utility functions for plotting
 * Hyperparameters
 * **Helper Functions**
 * The neural network models
 * A random planner
 * **A planner based on the cross-entropy method**
 * The main training loop, which coordinates rollouts, updating the replay buffer, and network updates

In [2]:
import time
from typing import Sequence, List

import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import torch
import tqdm
from gymnasium.utils.save_video import save_video
from torch import nn, optim
import random
from sklearn.preprocessing import StandardScaler

OUTPUT_FOLDER = DATA_ROOT / "exercise_4" / time.strftime("%Y-%m-%d_%H-%M")
OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)

# Set random seeds
SEED = 123
np.random.seed(SEED)
torch.manual_seed(SEED)


# Required to display matplotlib figures as cell output
%matplotlib inline

def plot_rewards(
        *reward_curves: np.ndarray,
        colors: Sequence[str],
        labels: Sequence[str],
        title: str = "Policy Performance during Training",
        ylabel: str = "Reward",
) -> None:
    plt.figure()
    for reward_curve, color, label in zip(reward_curves, colors, labels):
        plt.plot(reward_curve, color=color, label=label)
    plt.title(title)
    plt.xlabel("Iterations")
    plt.ylabel(ylabel)
    plt.legend()


def save_current_figure(save_name: str) -> None:
    plt.savefig(str(OUTPUT_FOLDER / f"{save_name}.png"))

### Set hyperparameters

In [ ]:
# In Google Colab, this code block renders as a form where hyperparameters can be adjusted.

# World model hyperparameters
# @markdown Number of random trajectories to collect
NUM_RAND_TRAJECTORIES = 10  # @param {type: "integer"}
# @markdown Dynamic Model learning rate
ENV_LEARNING_RATE = 3e-4  # @param {type: "number"}
# @markdown Reward Model learning rate
REW_LEARNING_RATE = 3e-4  # @param {type: "number"}
# @markdown Number of transitions to sample from the replay buffer for each update:
BATCH_SIZE = 2000  # @param {type: "integer"}
# @markdown Training Iterations:
TRAIN_ITER_MODEL = 50  # @param {type: "integer"}

# Controller hyperparameters
# @markdown Prediction horizon of the controller
HORIZON_LENGTH = 15  # @param {type: "integer"}
# @markdown Number of action sequences/trajectories to sample from the controller
NUM_ACTIONS_SEQUENCES = 100  # @param {type: "integer"}

# Training hyperparameters
# @markdown Number of outer training iterations
NUM_ITERATIONS = 100  # @param {type: "integer"}
# @markdown Number of steps/inner iterations
NUM_STEPS = 200  # @param {type: "integer"}
# @markdown The planner to use. "cem" or "random"
PLANNER = "cem"  # @param ["cem", "random"] {type: "string"}
# @markdown The number of iterations to optimize the cross entropy method for
CEM_OPTIMIZATION_ITERS = 5  # @param {type: "integer"}

# Collect Random Trajectories To Train A Dynamics Model and Reward Model (Experience Collection)

First, we randomly interact with the environment to produce a batch of experiences

$$D = \{s_t,  s_{t+1}, r_t, a_t\}_{t\in[1,N]}$$

In [3]:
def gather_random_trajectories(env: gym.Env) -> List[List[np.ndarray]]:
    """
    Collect random trajectories from an environment.

    This function runs a given number of random trajectories in the specified environment.
    It collects data about states, next states, rewards, and actions for each step in the trajectories.
    This data can be used to train models in a supervised manner.

    Args:
        env (gym.Env): The environment to run the trajectories in.

    Returns:
        list: A list containing the state, next state, reward, and action for each step.

    """
    rewards = []  # To store rewards for each game
    observations = []  # To store observations for each game
    next_observations = []  # To store next observations for each game
    actions = []  # To store actions for each game

    for _ in tqdm.tqdm(range(NUM_RAND_TRAJECTORIES), desc="Gathering Random Trajectories"):
        obs, _ = env.reset()  # Reset environment to start state
        done = False

        while not done:
            sampled_action = env.action_space.sample()  # Sample a random action
            new_obs, reward, terminated, truncated, _ = env.step(sampled_action)  # Apply action
            done = terminated or truncated

            # Append the observed data to the dataset
            observations.append(obs)
            next_observations.append(new_obs)
            rewards.append(reward)
            actions.append(sampled_action)

            obs = new_obs  # Update the current observation

    # create a transposed dataset for easier access
    data = [observations, next_observations, rewards, actions]
    data = list(map(list, zip(*data)))

    # Calculate and print statistics
    mean_reward = np.round(np.sum(rewards) / NUM_RAND_TRAJECTORIES, 2)
    max_reward = np.round(np.max(rewards), 2)
    avg_steps = np.round(len(rewards) / NUM_RAND_TRAJECTORIES)
    print(f"Random trajectory statistics:\n "
          f"Mean R: {mean_reward}, Max R: {max_reward}, Avg Steps: {avg_steps}")
    return data

# Deep Dynamics/Reward Models With Feed Forward NNs

A dynamics model takes in the state $s_t$ and action $a_t$ at the current time step and predicts the state $s_{t+1}$ at the next time step.  

**However this function can be difficult to learn when the states $s_t$ and $s_{t+1}$ are too similar and the action has seemingly littl eeffect on the output; this difficulty becomes more pronouncedas the time between states $\Delta t$ becomes smaller and the state differences do not indicate the underlying dynamics well. Thus we typically a function that predict the differences to the next state. i.e. $\Delta s_{t+1} = s_{t+1} - s_t = f(s_t,a_t)$. This allows us to learn more accurate models in practice.**

Similarly, a reward model takes in the state $s_t$ and action$a_t$ at the current time step and predicts reward $r_{t}$ that we can obtain based on that action.  

In the following block, we define the Feedforward Network. For each instance of this network, the input is a state and action at current time step $s_t$, $a_t$ respectively. The output is a reward for the reward model, and the difference to the next step for the dynamics model.
For NN training, we normally feed data in a mini-batch manner. Since each state in current task is a $1$st order image tensor (concatenation of $s_t$ and $a_t$), the model expects input to be a $2$nd order tensor with shape (mini-batch, state_dim + action_dim). And the output is a 2nd order tensor with shape (mini-batch, state_dim).

In [4]:
class FeedforwardModel(nn.Module):
    """
    Model that predict the difference to next state, given the current state and action
    """

    def __init__(self, input_dim: int, output_dim: int):
        """
        input_dim: Input dimension. Usually state_dim + action_dim
        output_dim: Output dimension. state_dim / reward dim
        """
        super(FeedforwardModel, self).__init__()

        # build a multi-layer perceptron
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(num_features=512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(num_features=256),
            nn.ReLU(),
            nn.Linear(256, output_dim)
        )

    def forward(self, x):
        return self.mlp(x.float())


# Training The Dynamics And Reward Models (3 Pts)
We can now train our models $f_\theta$ and $r_\theta$ in a supervised fashion to minimize an MSE loss over our experience batch by stochastic gradient descent:

$$L_{Dynamics} = \frac{1}{|D|}\sum_{s_t,a_t,s_{t+1}\in D}||\Delta s_{t+1}- f_\theta(s_t, a_t)||^2$$

$$L_{Reward} = \frac{1}{|D|}\sum_{s_t,a_t,r_{t}\in D}||r_{t}- r_\theta(s_t, a_t)||^2$$

-----
In practice, it’s helpful to normalize the target of a neural network.  So in the code, we’ll train the network to predict a normalized version of the change in state, as in

$$L_{Dynamics} = \frac{1}{|D|}\sum_{s_t,a_t,s_{t+1}\in D}||\text{normalize}(\Delta s_{t+1})- f_\theta(s_t, a_t)||^2$$

Similarly, we’ll train the network to predict a normalized version of the reward, as in

$$L_{Reward} = \frac{1}{|D|}\sum_{s_t,a_t,r_{t}\in D}||\text{normalize}(r_{t})- r_\theta(s_t, a_t)||^2$$



------
Since $f_{\theta}$ is trained to predict the normalized state difference, you generate thenext prediction with
$$
\hat{\mathbf{s}}_{t+1}=\mathbf{s}_{t}+\text { Unnormalize }\left(f_{\theta}\left(\mathbf{s}_{t}, \mathbf{a}_{t}\right)\right)
$$ \\

You generate the reward with $$
\hat{\mathbf{r}}_{t}=\text { Unnormalize }\left(r_{\theta}\left(\mathbf{s}_{t}, \mathbf{a}_{t}\right)\right)
$$

## Task 1: Training the Dynamics and Reward Models (5 Pts)
We will use the mean squared error (MSE) loss as shown in the equations above:
- implement the MSE loss in the *model_mse* function (1 pts)

To train the dynamics and reward models, we have to properly prepare the training data. Implement the following steps in the *train_dyna_model*
- split the dataset into training (80%) and testing (20%) data by converting the dataset into a proper list and slicing it using the *prepare_dataset* function (1 pts)
- randomly shuffle the training data (1 pts)
- create the input and output data for the dynamics and reward model. Think what the inputs for each of these networks are. Do this procedure for both train and validation data. (2 pts)


** The places you have to fill in your code are marked **

**Note** We use scikit learn's [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to perform normalization and unnormalization steps throughout this assignment.

In [6]:
def model_mse(y_true, y_pred, device):
    """
    Compute the MSE (Mean Squared Error) between y_truth and y_pred
    Args:
        y_true: ground truth values (batch_size x data_dimension)
        y_pred: predicted values (batch_size x data_dimension)
        device: device specification for pytorch ('cuda' / 'cpu)

    Returns: The MSE loss between y_true and y_pred

    """
    ### Your code starts here ###
    import torch.nn.functional as F
    y_true = torch.tensor(y_true, device=device)
    y_pred = torch.tensor(y_pred, device=device)

    loss = F.mse_loss(y_true, y_pred)
    return loss
    ### Your code ends here ###


def prepare_dataset(dataset):
    """
    Prepare the dataset for training and validation.

    This function processes the dataset to create inputs and outputs for the dynamics and reward models.
    It efficiently concatenates observations and actions, and computes the required labels.

    Args:
        dataset (list of tuples): The dataset containing observations, next observations, rewards, and actions.

    Returns:
        tuple: Numpy arrays for X, y_reward, y_next_state, and y_diff.
    """
    ### Your code starts here ###

    # Unpack the dataset into separate lists
    obs, next_obs, rewards, actions = zip(*dataset)

    # Convert lists to numpy arrays for efficient operations
    obs = np.array(obs)
    next_obs = np.array(next_obs)
    rewards = np.array(rewards)
    actions = np.array(actions)

    # Concatenate observations and actions
    X = np.concatenate([obs, actions], axis=1)


    # Compute y_diff
    y_diff = next_obs - obs

    ### Your code ends here ###
    return X, rewards, next_obs, y_diff


In [5]:
def train_dynamics_models(dataset, env_model, rew_model, device):
    """
    Train the two models that predict the next state and the expected reward
    Args:
        dataset: Random dataset to train the dynamics models on
        env_model: Model that predicts the next state
        rew_model: Model that predicts the reward
        device:

    Returns:

    """

    env_optimizer = optim.Adam(env_model.parameters(), lr=ENV_LEARNING_RATE)
    rew_optimizer = optim.Adam(rew_model.parameters(), lr=REW_LEARNING_RATE)

    # Split the dataset into randomly assigned train(80%) and test(20%) splits
    ### Your code starts here ###
    train_length = len(dataset) * 0.8
    valid_length = len(dataset) - train_length

    D_train, D_valid = torch.utils.data.random_split(dataset, [train_length, valid_length])
    ### Your code ends here ###

    X_train, y_rew_train, y_next_state_train, y_diff_train = prepare_dataset(D_train)
    X_valid, y_rew_valid, y_next_state_valid, y_diff_valid = prepare_dataset(D_valid)

    ### Normalize the inputs and outputs for nicer NN training ###


    input_scaler = StandardScaler()
    env_output_scaler = StandardScaler()
    rew_output_scaler = StandardScaler()

    # Normalize the features X, y_diff, y_rew by removing the mean and scaling to unit variance, i.e., normalizing to N(0,1).
    # Fit the scalers on the training data and use them to transform the validation data and the planner in the main training loop.
    ### Your code starts here ###
    X_train = input_scaler.fit_transform(X_train)
    y_diff_train = env_output_scaler.fit_transform(y_diff_train)
    y_rew_train = rew_output_scaler.fit_transform(y_rew_train)

    X_valid = input_scaler.transform(X_valid)
    y_diff_valid = env_output_scaler.transform(y_diff_valid)
    y_rew_valid = rew_output_scaler.transform(y_rew_valid)
    ### Your code end here ###

    # store all the scalers/normalizers in a list for later planning
    normalizers = (input_scaler, env_output_scaler, rew_output_scaler)

    losses_env = []
    losses_rew = []
    valid_env_loss = 0
    valid_rew_loss = 0

    # go through max_model_iter supervised iterations
    for iteration in tqdm.tqdm(range(TRAIN_ITER_MODEL)):
        # create mini batches of size batch_size
        for batch in range(0, len(X_train), BATCH_SIZE):
            if len(X_train) > batch + BATCH_SIZE:  # if there are enough examples left
                X_batched = X_train[batch:batch + BATCH_SIZE]

                y_diff_batched = y_diff_train[batch:batch + BATCH_SIZE]
                y_rew_batched = y_rew_train[batch:batch + BATCH_SIZE]

                # Add gaussian noise with mean 0 and variance 0.0001 as in the paper
                X_batched += np.random.normal(loc=0, scale=0.001, size=X_batched.shape)

                # Optimization of the 'env_model' and 'rew_model' networks
                env_optimizer.zero_grad()
                rew_optimizer.zero_grad()

                # forward pass of the models to compute current outputs
                pred_state = env_model(torch.tensor(X_batched).to(device))
                pred_rew = rew_model(torch.tensor(X_batched).to(device))

                # compute the MSE loss
                loss_env = model_mse(y_diff_batched, pred_state, device)
                loss_rew = model_mse(y_rew_batched, pred_rew, device)

                # backward pass and optimization step
                loss_env.backward()
                env_optimizer.step()
                loss_rew.backward()
                rew_optimizer.step()

                loss_env = loss_env.cpu().detach().numpy()
                loss_rew = loss_rew.cpu().detach().numpy()
                if iteration == (TRAIN_ITER_MODEL - 1):
                    # take losses of the last iteration
                    losses_env.append(loss_env)
                    losses_rew.append(loss_rew)
        if iteration % 10 == 0 or iteration == TRAIN_ITER_MODEL - 1:
            # Evalute the models every 10 iterations and print the losses
            env_model.eval()
            rew_model.eval()

            pred_state = env_model(torch.tensor(X_valid).to(device))
            pred_rew = rew_model(torch.tensor(X_valid).to(device))
            valid_env_loss = model_mse(y_diff_valid, pred_state, device).cpu().detach().numpy()
            valid_rew_loss = model_mse(y_rew_valid, pred_rew, device).cpu().detach().numpy()

            print(f" Model Training Iteration: {iteration}, "
                  f"Validation Dynamics Loss: {valid_env_loss}, "
                  f"Validation Reward Loss: {valid_rew_loss}")

            # Set the models back to training mode
            env_model.train(True)
            rew_model.train(True)

    losses_env = np.mean(losses_env)
    losses_rew = np.mean(losses_rew)
    return losses_env, losses_rew, valid_env_loss, valid_rew_loss, normalizers

# Planning With Dynamics And Reward Models

Given the learned dynamics and reward models, we now want to select and execute actions that minimize a cost function (long term rewards). Ideally, you would calculate these actions by solving the following optimization:

$$
\mathbf{a}_{t}^{*}=\arg \max _{\mathbf{a}_{t: \infty}} \sum_{t^{\prime}=t}^{\infty} r_{\theta}\left(\hat{\mathbf{s}}_{t^{\prime}}, \mathbf{a}_{t^{\prime}}\right) \text { where } \hat{\mathbf{s}}_{t^{\prime}+1}=\hat{\mathbf{s}}_{t^{\prime}}+f_{\theta}\left(\hat{\mathbf{s}}_{t^{\prime}}, \mathbf{a}_{t^{\prime}}\right)
$$

However, solving the above equation is impractical for two reasons:
- planning over an infinite sequence of actions is impossible and
- the learned dynamics model is imperfect, so using it to plan in such an open-loop manner will lead to accumulating errors over time and planning far into the future will become very inaccurate.

Instead, we will solve the following gradient-free optimization problem:
$$
\mathbf{A}^{*}=\arg \max _{\left\{\mathbf{A}^{(0)}, \ldots, \mathbf{A}^{(K-1)}\right\}} \sum_{t^{\prime}=t}^{t+H-1} r_{\theta}\left(\hat{\mathbf{s}}_{t^{\prime}}, \mathbf{a}_{t^{\prime}}\right) \text { s.t. } \hat{\mathbf{s}}_{t^{\prime}+1}=\hat{\mathbf{s}}_{t^{\prime}}+f_{\theta}\left(\hat{\mathbf{s}}_{t^{\prime}}, \mathbf{a}_{t^{\prime}}\right)
$$
in which $\mathbf{A}^{(k)}=\left(a_{t}^{(k)}, \ldots, a_{t+H-1}^{(k)}\right)$ are each a random action sequence of length $H$.


## Planner 1: Random Shooting Based Planner

We will now use a simple random shooting method to solve the following gradient-free optimization problem :
$$
\mathbf{A}^{*}=\arg \min _{\left\{\mathbf{A}^{(0)}, \ldots, \mathbf{A}^{(K-1)}\right\}} \sum_{t^{\prime}=t}^{t+H-1} r_{\theta}\left(\hat{\mathbf{s}}_{t^{\prime}}, \mathbf{a}_{t^{\prime}}\right) \text { s.t. } \hat{\mathbf{s}}_{t^{\prime}+1}=\hat{\mathbf{s}}_{t^{\prime}}+f_{\theta}\left(\hat{\mathbf{s}}_{t^{\prime}}, \mathbf{a}_{t^{\prime}}\right)
$$
in which $\mathbf{A}^{(k)}=\left(a_{t}^{(k)}, \ldots, a_{t+H-1}^{(k)}\right)$ are each a random action sequence of length $H$.

**Random Shooting**: The simplest gradient-free optimizer simply generates $N$ independent random action sequences $\left\{A_{0} \ldots A_{N}\right\}$, where each sequence $A_{i}=\left\{a_{t}^{i} \ldots a_{t+H-1}^{i}\right\}$ is of length $H$ action. Given a reward function $r(s, a)$ that defines the task, and given future state predictions $\hat{s}_{t+1}=f_{\theta}\left(\hat{s}_{t}, a_{t}\right)+\hat{s}_{t}$ from the learned dynamics model $f_{\theta}$, the optimal action sequence $A_{i^{*}}$ is selected to be the one corresponding to the sequence with highest predicted return: $i^{*}=\arg \max _{i} \sum_{t^{\prime}=t}^{t+H-1} r\left(\hat{s}_{t^{\prime}}, a_{t^{\prime}}^{i}\right) .$

In [7]:
def random_control(env_model, rew_model, observation, sample_action, normalizers,
                   device):
    """
    Use a random-sampling method to generate action sequences.
    The action for the first step that leads to the highest predicted sequence reward is returned.
    Args:
        env_model: The transition model
        rew_model: The reward model
        observation: The observation last seen from the real physical environment
        sample_action: The function which is used to sample the action. Usually env.action_space.sample.
        normalizers: Normalizers for the input and output of the models
        device: The device to run the models on ('cuda' / 'cpu')

    Returns: The action to take and the predicted sum of rewards

    """
    # Prepare the models for evaluation
    env_model.eval()
    rew_model.eval()

    input_scaler, env_output_scaler, rew_output_scaler = normalizers

    # Initialize an array with repeated observations for batch processing
    batch_obs = np.array([observation for _ in range(NUM_ACTIONS_SEQUENCES)])

    # Array to store cumulative rewards for all action sequences
    cumulative_rewards = np.zeros((NUM_ACTIONS_SEQUENCES, 1))

    # Sampling action sequences for each trajectory
    sampled_action_sequences = []
    for _ in range(NUM_ACTIONS_SEQUENCES):
        sampled_action_sequence = [sample_action() for _ in range(HORIZON_LENGTH)]
        sampled_action_sequences.append(sampled_action_sequence)
    sampled_action_sequences = np.array(sampled_action_sequences)

    for t in range(HORIZON_LENGTH):
        # select action for time step t for each sequence
        sampled_actions = sampled_action_sequences[:, t, :]

        # scale the input
        models_input = input_scaler.transform(np.concatenate([batch_obs, sampled_actions], axis=1))
        # compute the next state and reward for the state-action pair
        pred_obs = env_model(torch.tensor(models_input).to(device))
        pred_rew = rew_model(torch.tensor(models_input).to(device))

        # unnormalize/descale and add previous observation
        pred_obs = env_output_scaler.inverse_transform(pred_obs.cpu().detach().numpy())
        batch_obs = pred_obs + batch_obs

        # sum of the expected rewards
        cumulative_rewards += pred_rew.cpu().detach().numpy()

    # Identify the sequence with the highest cumulative reward
    arg_best_reward = np.argmax(cumulative_rewards)
    best_sum_reward = cumulative_rewards[arg_best_reward].squeeze()
    # take the first action of this sequence
    first_actions = sampled_action_sequences[:, 0, :]
    best_action = first_actions[arg_best_reward]

    env_model.train(True)
    rew_model.train(True)
    return best_action, best_sum_reward

# Cross Entropy Method (CEM) for Planning
The random shooting approach has been shown to achieve success on continuous control tasks with learned models, but it has numerous drawbacks: it scales poorly with the dimension of both the planning horizon and the action space, and it often is insufficient for achieving high task performance since a sequence of actions sampled at random often does not directly lead to meaningful behavior. Therefore, the Cross Entropy Method is a favorable planning approach in model based reinforcement learning.

We already got to know to CEM in the stochastic search homework. However, here, we consider a different application.

We will use CEM to solve the following gradient-free optimization problem :
$$
\mathbf{A}^{*}=\arg \min _{\left\{\mathbf{A}^{(0)}, \ldots, \mathbf{A}^{(K-1)}\right\}} \sum_{t^{\prime}=t}^{t+H-1} r_{\theta}\left(\hat{\mathbf{s}}_{t^{\prime}}, \mathbf{a}_{t^{\prime}}\right) \text { s.t. } \hat{\mathbf{s}}_{t^{\prime}+1}=\hat{\mathbf{s}}_{t^{\prime}}+f_{\theta}\left(\hat{\mathbf{s}}_{t^{\prime}}, \mathbf{a}_{t^{\prime}}\right)
$$
in which $\mathbf{A}^{(k)}=\left(a_{t}^{(k)}, \ldots, a_{t+H-1}^{(k)}\right)$ are each a random action sequence of length $H$

**Cross-Entropy Method** (`CEM`) is an optimization algorithm applicable to problems that are both **combinatorial** and **continuous**, which is our case: find the best performing sequence of actions.

The Cross-entropy method (CEM) approach, begins like the random shooting approach, but then does this sampling for multiple iterations $m \in\{0 \ldots M\}$ at each time step. The top $J$ highest-scoring action sequences from each iteration are used to update and refine the mean and variance of the sampling distribution for the next iteration, as follows:

\begin{aligned}
A_{i} &=\left\{a_{0}^{i} \ldots a_{H-1}^{i}\right\} \text {, where } a_{t}^{i} \sim \mathcal{N}\left(\mu_{t}^{m}, \Sigma_{t}^{m}\right) \forall i \in N, t \in 0 \ldots H-1 \\
A_{\text {elites }} &=\operatorname{sort}\left(A_{i}\right)[-J:] \\
\mu_{t}^{m+1} &=  \operatorname{mean}\left(A_{\text {elites }}\right) \quad \forall t \in 0 \ldots H-1 \\
\Sigma_{t}^{m+1} &= \operatorname{var}\left(A_{\text {elites }}\right)\quad  \forall t \in 0 \ldots H-1
\end{aligned}

After $M$ iterations, the optimal actions are selected to be the resulting mean of the action distribution.
Note that, since our model is imperfect and things will never go perfectly according to plan, we adopt a model predictive control (MPC) approach.
The MPC planner replans at every time step similar to previous section with random shooting planner. Students may refer to [this paper](/https://arxiv.org/pdf/1909.11652.pdf).

## Task 2: Implementing the CEM for Planning (7 Pts)
In the following task you will implement the CEM following the standard procedure (5 Pts). In each optimization step, you need to
- sample a sequence of actions $A_{i}$ from the Gaussian distribution from the iteration before
- prepare the data for inputting to the dynamics and rewards model by standardizing it
- perform the inference of the reward model to obtain a reward prediction
- perform the inference via the dynamics model to obtain the observation difference prediction
- unnormalize the predicted observation difference
- obtain the observation of the next step by adding the predicted observation to the current observation
- sum the predicted reward to the rewards from the steps before

After each optimization step (2 Pts):
- obtain the top 10 action sequences $A_{i}$  which lead to the highest cumulative reward
- update the search distribution by calculating the sample mean and the sample standard deviation using the elite samples
**NOTE:** We only consider the standard deviation of each dimension of the search space, i.e., we use an isotropic Gaussian distribution.

We have provided you with comments guiding you through the parts you need to implement.

In [ ]:
def cem_control(env_model, rew_model, observation, sample_action, normalizers,
                device):
    """
    Use a cross entropy method to generate action sequences.
    The action for the first step that leads to the highest predicted sequence reward is returned.
    Args:
        env_model: The transition model
        rew_model: The reward model
        observation: The observation last seen from the real physical environment
        sample_action: The function which is used to sample the action. Usually env.action_space.sample.
        normalizers: Normalizers for the input and output of the models
        device: The device to run the models on ('cuda' / 'cpu')

    Returns: The action to take and the predicted sum of rewards

    """
    env_model.eval()
    rew_model.eval()

    input_scaler, env_output_scaler, rew_output_scaler = normalizers

    # Batch-wise sampling of action sequences. Each sequence is of length H.
    sampled_action_sequences = []
    for _ in range(NUM_ACTIONS_SEQUENCES):
        sampled_action_sequence = [sample_action() for _ in range(HORIZON_LENGTH)]
        sampled_action_sequences.append(sampled_action_sequence)
    sampled_action_sequences = np.array(sampled_action_sequences)
    action_dim = sampled_action_sequences.shape[-1]

    ### Your code starts here ###

    # Calculate the mean and standard deviation of the sampled action sequences to get an initial estimate of the
    # search distribution
    action_mean = ...
    action_std_dev = ...

    first_sampled_actions = []
    cumulative_rewards = None
    for _ in range(CEM_OPTIMIZATION_ITERS):
        """
        In this section you implement an MPC with CEM method.  In each optimization iteration,
        you sample actions sequences from a normal distribution, Calculate the cost for each
        sequence using the learned dynamics and reward models. We will use MPC similar to the
        previous section to re-plan at every timestep.
        """
        # Sample from a Gaussian Using The Means and Standard Deviations
        sampled_action_sequences = ...

        # Initialize an array with repeated observations for batch processing
        cumulative_rewards = np.zeros((NUM_ACTIONS_SEQUENCES, 1))
        batch_obs = ...

        first_sampled_actions = sampled_action_sequences[:, 0, :]
        for t in range(HORIZON_LENGTH):
            # sample actions for each sequence and scale the input
            sampled_actions = ...
            models_input = input_scaler.transform(np.concatenate([batch_obs, sampled_actions], axis=1))

            # compute the differences to the next state using the dynamics and reward model
            pred_obs = ...
            pred_rew = ...

            # unnormalize/descale and add previous observation
            pred_obs_unnormalized = ...
            batch_obs = pred_obs_unnormalized + batch_obs

            # sum the expected rewards
            cumulative_rewards += pred_rew.cpu().detach().numpy()

        # Select Top K Action Sequences (lets call them elite_sequences) that gave the highest cumulative reward
        elite_sequences = ...

        # Update the mean and variances of the search distribution using the elite (topk) action sequences
        action_mean, action_std_dev = ...

    ### Your code ends here ###

    # Pick the first action of the sequence with the highest cumulative reward
    arg_best_reward = np.argmax(cumulative_rewards)
    best_sum_reward = cumulative_rewards[arg_best_reward].squeeze()

    # take the first action of this sequence
    best_action = first_sampled_actions[arg_best_reward]
    # you can also choose the best action as the mean of the elites from last optimization iteration.
    # However, here we do a greedy step in the end where the best among the elites from last iteration is chosen.

    env_model.train(True)
    rew_model.train(True)

    return best_action, best_sum_reward

# Task 3: What is a Model Predictive Controller (MPC)? (1 Pts)

Since our dynamics and reward models can be imperfect and things will never go perfectly ac-ording to plan, we adopt a model predictive control (MPC) approach. Explain in few lines whats the basic priniciple behind an MPC.

# Main Loop (2 Pts)

Start with the default hyperparameters and run the main MBRL loop with the two planners. Try out other hyperparameters and compare the reward plots of both planners for these parameters. Briefly (2-3 sentences) describe your observations.

\begin{aligned}
&\overline{\text { Algorithm } \mathbf{1} \text { Model-based Reinforcement Learning }} \\
&\hline \text { 1: gather dataset } \mathcal{D}_{\text {RAND }} \text { of random trajectories } \\
&\text { 2: initialize empty dataset } \mathcal{D}_{\text {RL }} \text {, and randomly initialize } f_{\theta} \\
&  \text{ 3: } \textbf{for} \text{ iter=1 to max_iter } \textbf{do}  \\
&\quad  \quad \quad \textbf {Model Learning } \\
&\text { 4:} \quad  \quad \text{ train } f_{\theta}(\mathbf{s}, \mathbf{a}) \text{ and } r_{\theta}(\mathbf{s}, \mathbf{a}) \text { by performing gradient descent on MSE Loss }  \\
& \quad \quad  \quad \text { using } \mathcal{D}_{\text {RAND }} \text { and } \mathcal{D}_{\text {RL }} \\
&\quad  \quad \quad \textbf {Planning and More Experience Collection } \\
&\text { 5: } \quad  \quad  \textbf { for } t=1 \text { to } T \textbf { do } \\
&\text { 6: } \quad \quad \quad \quad \text { get agent's current state } \mathbf{s}_{t} \\
&\text { 7: } \quad \quad \quad \quad \text { use } f_{\theta} \text { and } r_{\theta} \text { to estimate optimal action sequence } \mathbf{A}_{t}^{(H)} \\
&\text { 8: } \quad \quad \quad \quad \text { execute first action } \mathbf{a}_{t} \text { from selected action sequence } \\
&\text { 9: } \quad \quad \quad \quad\text { Add } \{s_t,  s_{t+1}, r_t, a_t\} \text { to } \mathcal{D}_{\text {RL }} \\
&\text { 10: } \quad \text { end for } \\
&\text { 11: end for } \\
&\hline
\end{aligned}

In [ ]:
def train():
    device = 'cpu'  # 'cuda' or 'cpu'
    environment_str = "Pendulum-v1"  # "InvertedPendulum"

    video_folder = DATA_ROOT / "exercise_4" / time.strftime("%Y-%m-%d_%H-%M")

    env = gym.make(environment_str, render_mode="rgb_array_list")
    _ = env.reset()

    # Step 1:  gather the dataset of random sequences
    dataset = gather_random_trajectories(env=env)  # initialize the dataset randomly

    return_list = []
    train_env_losses = []
    train_rew_losses = []
    validation_env_losses = []
    validation_rew_losses = []

    # Step 2: Initialize the models
    in_features = env.unwrapped.action_space.shape[0] + env.unwrapped.observation_space.shape[0]
    env_model = FeedforwardModel(input_dim=in_features,
                                 output_dim=env.unwrapped.observation_space.shape[0]).to(device)
    rew_model = FeedforwardModel(input_dim=in_features,
                                 output_dim=1).to(device)

    # Step 3: Iterate over model learning, planning and on-policy experience collection
    for n_iter in range(NUM_ITERATIONS):

        # supervised training of the current experience dataset
        train_env_loss, train_rew_loss, valid_env_loss, valid_rew_loss, standardizers = train_dynamics_models(dataset,
                                                                                                              env_model,
                                                                                                              rew_model,
                                                                                                              device)
        env = gym.make(environment_str, render_mode="rgb_array_list")

        returns = []

        num_transitions = 0
        p_bar = tqdm.tqdm(total=NUM_STEPS, desc="MPC Planner Steps: ")
        while num_transitions < NUM_STEPS:
            done = False
            env_return = 0
            pred_return = 0
            obs, _ = env.reset()
            while not done:

                # Execute the control to roll the sequences and pick the first action of the sequence with the higher reward
                planner_fn = cem_control if PLANNER == 'cem' else random_control
                action, pred_rew = planner_fn(env_model, rew_model, obs,
                                              env.action_space.sample,
                                              standardizers, device)

                # one step in the environment with the action returned by the controller
                new_obs, reward, terminated, truncated, _ = env.step(action)  # Apply action
                done = terminated or truncated

                ## Compute the reward using the reward model
                rew_model.eval()
                input_scaler, _, rew_output_scaler = standardizers
                models_input = input_scaler.transform([np.concatenate([obs, action])])
                pred_reward = rew_model(torch.tensor(models_input).to(device))
                unnorm_reward = rew_output_scaler.inverse_transform(pred_reward.cpu().detach().numpy())
                pred_return += unnorm_reward
                rew_model.train(True)

                # add the transition to the dataset
                dataset.append([obs, new_obs, reward, action])
                obs = new_obs
                env_return += reward
                num_transitions += 1
                p_bar.update(1)

                # if the environment is done, print some stats
                if done:
                    returns.append(env_return)
                    print(f"Observed/predicted return for current episode: {env_return}/{np.mean(pred_return)}")

        print(f"Average returns at iteration {n_iter}: {np.mean(returns)} ")
        return_list.append(np.mean(returns))
        train_env_losses.append(train_env_loss)
        validation_env_losses.append(valid_env_loss)
        train_rew_losses.append(train_rew_loss)
        validation_rew_losses.append(valid_rew_loss)

        # clean up and save video of last episode
        save_video(
            env.render(),
            video_folder=video_folder,
            episode_trigger=lambda x: x % 250 == 0,
            name_prefix="mbrl",
            episode_index=NUM_ITERATIONS,
            fps=30,
        )
        env.close()
        p_bar.close()

    # Step 4: Do some plotting
    plot_rewards(np.array(return_list),
                 colors=["blue"],
                 labels=["MBRL"],
                 title=f"MBRL - {PLANNER}",
                 ylabel="Return")
    save_current_figure(f"MPC_Reward_{PLANNER}")

    plot_rewards(np.array(train_env_losses), np.array(validation_env_losses),
                 colors=["blue", "red"],
                 labels=["Train", "Validation"],
                 title=f"MBRL - {PLANNER}",
                 ylabel="Env Loss")
    save_current_figure(f"Env_Loss_{PLANNER}")

    plot_rewards(np.array(train_rew_losses), np.array(validation_rew_losses),
                 colors=["blue", "red"],
                 labels=["Train", "Validation"],
                 title=f"MBRL - {PLANNER}",
                 ylabel="Reward Loss")
    save_current_figure(f"Rew_Loss_{PLANNER}")
    plt.show()


train()
